In [1]:
import torch
import numpy as np
import os
import shutil
from datetime import datetime
import math
from math import exp
from random import seed
import random as rand
import matplotlib.pyplot as plt
import pickle 

import sys
sys.path.append('../common/')
from utils import *
from optimisation import *
from loo_cv import *

In [2]:
seed=0
torch.manual_seed(seed)

This notebooks computes the test error, the 1d interpolatione error on 3 test cases and the 2d interpolation error in 3 test cases, both in uniform mesh and non-uniform mesh.

In [3]:
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.nn.functional as F           # layers, activations and more
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.
from torch.jit import script, trace       # hybrid frontend decorator and tracing jit
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR

In [4]:
class Net(nn.Module):
    def __init__(self,N=10):
            super(Net, self).__init__()
            
            nInputs = int(N*N/2-N/2)
            
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(in_features=nInputs, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=64),
                nn.ReLU(),
                nn.Linear(in_features=64, out_features=32),
                nn.ReLU(),
                nn.Linear(in_features=32, out_features=16),
                nn.ReLU(),
                nn.Linear(in_features=16, out_features=1)
            )
            # map to positive
            
            
    def forward(self, x):
            logits = self.linear_relu_stack(x)
            return logits

# generate features

In [5]:
def upper_half(A):
    n = A.shape[0]
    return 1/A[np.triu_indices(n, k = 1)]

In [6]:
# network to be tested:
directory = "plots/"
if not os.path.exists(directory):
    os.makedirs(directory)

n=10
path =   '../network/results_sorted_1d_2d/best_model.pt'
model=Net()
model.load_state_dict(torch.load(path))
model.eval()

Net(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=45, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): ReLU()
    (8): Linear(in_features=32, out_features=16, bias=True)
    (9): ReLU()
    (10): Linear(in_features=16, out_features=1, bias=True)
  )
)

# checking condition number on test/train set

In [7]:
# interpolation two dimension uniform

In [9]:
# new interpolation two dimension 
import math
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import os
import shutil
from datetime import datetime
import math
from math import exp
from random import seed

from scipy.spatial import distance
import pickle
from rbf_tools import *
from two_dim import *

def phi(f, x, y):
    z = (1 + (f * np.linalg.norm(x-y)) ** 2) ** (-0.5)
    return z

In [12]:

def euclidean_distance(p1, p2):
    return np.sqrt(np.sum((p1 - p2) ** 2))

def find_nearest_neighbors2(points):
    num_points = points.shape[0]
    distances = np.full(num_points, np.inf)
    
    for i in range(num_points):
        for j in range(num_points):
            if i != j:
                distance = euclidean_distance(points[i], points[j])
                if distance < distances[i]:
                    distances[i] = distance
    
    return distances


def find_nearest_neighbors(points):
    num_points = points.shape[0]
    
    distance_matrix =cdist(points,points) 
    np.fill_diagonal(distance_matrix,np.inf)
    
    distances = np.zeros(num_points)
    
    for i in range(num_points):
        distances[i] = np.min(distance_matrix[i,:])
        
    return distances

# Function to compute the Euclidean distance between two points
def dist(p1, p2):
    return math.hypot(p1[0] - p2[0], p1[1] - p2[1])

# Function to check if a point is inside a circle
def is_inside_circle(p, c):
    return dist(p, c[:2]) <= c[2]

# Function to compute the circle from 3 points
def circle_from_3_points(p1, p2, p3):
    A = p2[0] - p1[0]
    B = p2[1] - p1[1]
    C = p3[0] - p1[0]
    D = p3[1] - p1[1]
    E = A * (p1[0] + p2[0]) + B * (p1[1] + p2[1])
    F = C * (p1[0] + p3[0]) + D * (p1[1] + p3[1])
    G = 2 * (A * (p3[1] - p2[1]) - B * (p3[0] - p2[0]))
    
    if G == 0:  # Collinear points
        return None
    
    cx = (D * E - B * F) / G
    cy = (A * F - C * E) / G
    r = dist((cx, cy), p1)
    
    return (cx, cy, r)

# Function to compute the circle from 2 points
def circle_from_2_points(p1, p2):
    cx = (p1[0] + p2[0]) / 2
    cy = (p1[1] + p2[1]) / 2
    r = dist(p1, p2) / 2
    return (cx, cy, r)

# Recursive function to find the minimum enclosing circle
def welzl(P, R, n):
    if n == 0 or len(R) == 3:
        if len(R) == 0:
            return (0, 0, 0)
        elif len(R) == 1:
            return (R[0][0], R[0][1], 0)
        elif len(R) == 2:
            return circle_from_2_points(R[0], R[1])
        else:
            return circle_from_3_points(R[0], R[1], R[2])
    
    idx = rand.randint(0, n - 1)
    p = P[idx]
    P[idx], P[n - 1] = P[n - 1], P[idx]
    
    d = welzl(P, R, n - 1)
    
    if is_inside_circle(p, d):
        return d
    
    return welzl(P, R + [p], n - 1)

# Function to find the minimum enclosing circle
def find_min_circle(points):
    P = points[:]
    #random.shuffle(P)
    return welzl(P, [], len(P))

In [13]:
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import cdist
import time

def two_interpolation(model,testcase,no_testcase,N,oversampling,a,b,c,d,n):
    
    p_ref=np.array([0,0])       
    x=get_structured_points_modified(a,b,c,d,N)
    evall=get_structured_points_modified(a,b,c,d,oversampling*N)
    no_interpolation_points= x.shape[0]
    no_evaluation_points= evall.shape[0]
     
    distance =cdist(evall,x)

    closest = np.argpartition(distance, 10, axis=1)
    scaled_training_set_flat = []
    x_coords = []
    
    eps_final_hardy=np.zeros((no_evaluation_points,1))
    eps_final_franke=np.zeros((no_evaluation_points,1))
    eps_final_mfranke=np.zeros((no_evaluation_points,1))
    eps_final_rippa=np.zeros((no_evaluation_points,1))

    nn_time_accum = 0
    hardy_time_accum = 0
    franke_time_accum = 0
    mfranke_time_accum = 0
    rippa_time_accum = 0
    
    # generate features
    for i in range(no_evaluation_points):
        x_local=np.zeros((n,2))
        for j in range(n):
            x_local[j]=x[closest[i][j]]

        x_local = sorted( x_local, key = lambda x: np.linalg.norm(x - p_ref ) )#change for sorted inputs
        x_local = np.reshape(x_local, (10,2)) 
        x_coords.append(x_local)
        x_axis=x_local.reshape(10,2)
        #x_axis = sorted( x_axis, key = lambda x: np.linalg.norm(x - p_ref ) )#change for sorted inputs
        #x_axis = np.reshape(x_axis, (10,2)) 

        start = time.process_time()
        nearest_distances = find_nearest_neighbors(x_axis)
        d=np.sum(nearest_distances)/n        
        eps_final_hardy[i]=1/(0.815*d)
        hardy_time = (time.process_time() - start) 
        hardy_time_accum += hardy_time

        
        start = time.process_time()  
        x_axis_list=x_axis.tolist()
        circle = find_min_circle(x_axis_list)
        d_time = (time.process_time() - start)

        start = time.process_time()
        eps_final_franke[i]=0.8*(10**(1/2))/(2 * circle[2])
        franke_time = (time.process_time() - start) + d_time
        franke_time_accum += franke_time

        start = time.process_time()
        eps_final_mfranke[i]=0.8*(10**(1/4))/(2 * circle[2])
        mfranke_time = (time.process_time() - start) + d_time
        mfranke_time_accum += mfranke_time

        start = time.process_time()
        #compute shape parameter from Rippa approach       
        rhs_rippa = np.array([testcase(x[0],x[1]) for x in x_axis])
        #eps_v = np.linspace(0.001,200,600)
        #eps_v = np.linspace(0.001,1000,24)
        eps_v = [0.001, 0.002, 0.005, 0.0075, 0.01, 0.02, 0.05, 0.075, 0.1, 0.2, 0.5, 0.75, 1.0,\
                 2.0, 5.0, 7.5, 10.0, 20.0, 50.0, 75.0, 100.0, 200.0, 500.0, 1000.0] # from python library

        best_eps = 0
        old_error = np.inf
        
        for eps in eps_v:
            error_r = rippa_cv(eps, x_axis, rhs_rippa)
            if error_r < old_error:
               best_eps = eps
               old_error = error_r
        eps_final_rippa[i]=best_eps
        rippa_time = (time.process_time() - start)
        rippa_time_accum+=rippa_time

        start = time.process_time()
        xxx=generate_distance_from_coordinates(x_axis)               
        training_set_distances_flatten = upper_half(xxx)
        scaled_training_set_flat.append(training_set_distances_flatten)
        nn_time_accum += (time.process_time() - start)

    start = time.process_time()
    scaled_training_set_flat = np.array(scaled_training_set_flat)
    scaled_training_set_tensor = torch.tensor(scaled_training_set_flat,dtype=torch.float)
    scaled_training_set_tensor.reshape((no_evaluation_points,int(n*n/2-n/2)))
    
    
    eps_final= model(scaled_training_set_tensor)
    eps_final_optimisation = eps_final.detach().numpy()
    nn_time_accum += (time.process_time() - start)

    error2=[0 for i in range(5)]
    
    for j in range(no_evaluation_points):
        for count,eps in enumerate([eps_final_rippa[j][0],eps_final_hardy[j][0],eps_final_franke[j][0],eps_final_mfranke[j][0],eps_final_optimisation[j][0]]):
        
            L =  get_int_matrix(x_coords[j],eps)
            cond = np.linalg.cond(L,'fro')
            final_error=compute_evaluation_error(x_coords[j],evall[j],testcase,phi,L,eps,n)
            error2[count]=error2[count]+final_error
        
    final_tot_error = [math.sqrt(err/no_evaluation_points) for err in error2]
    timings = [nn_time_accum, hardy_time_accum, franke_time_accum, mfranke_time_accum, rippa_time_accum]
    return final_tot_error, timings


In [14]:
nunsuccessful = 0
oversampling=2
a,c=0.0,0.0
b,d=1.0,1.0        

In [ ]:
n=10 # length of stencil
testcase1=lambda x,y,alpha=0.1: (1+np.exp(-1.0/alpha)-np.exp(-x/alpha)-np.exp((x-1.0)/alpha))*(1+np.exp(-1.0/alpha)-np.exp(-y/alpha)-np.exp((y-1.0)/alpha))
testcase2=lambda x,y,alpha=1.0: (1+np.exp(-1.0/alpha)-np.exp(-x/alpha)-np.exp((x-1.0)/alpha))*(1+np.exp(-1.0/alpha)-np.exp(-y/alpha)-np.exp((y-1.0)/alpha))
testcase3=lambda x,y: 0.75*np.exp(-(9.0*x-2.0)**2/4.0-(9.0*y-2.0)**2/4.0)+0.75*np.exp(-(9.0*x+1.0)**2/49.0-(9.0*y+1.0)**2/10.0)+0.5*np.exp(-(9.0*x-7.0)**2/4.0-(9.0*y-3.0)**2/4.0)-0.2*np.exp(-(9.0*x-4.0)**2-(9.0*y-7.0)**2)

no_testcase=1
nx=[20,40,60,80,100,120]

for testcase in [testcase1,testcase2,testcase3]:
    no_test=no_testcase
    x_vec=[]

    error_rippa=[]
    error_hardy=[]
    error_franke=[]
    error_mfranke=[]
    error_optimisation=[]
    
    for N in nx:
        final_error, timings=two_interpolation(model,testcase,no_testcase,N,oversampling,a,b,c,d,n)
        x_vec.append(N*N*oversampling*oversampling)
        error_rippa.append(final_error[0])
        error_hardy.append(final_error[1])
        error_franke.append(final_error[2])
        error_mfranke.append(final_error[3])
        error_optimisation.append(final_error[4])

        # Elements & Hardy & Frake & Modified Franke & Rippa & NN\\ [0.5ex] 
        print(f"{N} & {timings[1]:.1e} & {timings[2]:.1e} & {timings[3]:.1e}\
                & {timings[4]:.1e} & {timings[0]:.1e}  \\\\ ")
    print(error_rippa)
    print(error_hardy)
    print(error_franke)
    print(error_mfranke)
    print(error_optimisation)
    print('********')
    fig=plt.figure()
    plt.loglog(x_vec,error_rippa,color='b',marker='o')
    plt.loglog(x_vec,error_hardy,color='orange',marker='o')
    plt.loglog(x_vec,error_franke,color='g',marker='o')
    plt.loglog(x_vec,error_mfranke,color='purple',marker='o')
    plt.loglog(x_vec,error_optimisation,color='r',marker='o')
    
    plt.ylabel('Error', fontsize="12")
    plt.xlabel('# Evaluation points', fontsize="12")
    plt.legend(["Rippa","Hardy","Franke","Modified Franke","NN"],fontsize="12",  loc ="best")
    plt.savefig(directory+'testcase'+str(no_testcase)+'.png')  
    plt.close()
    
    no_testcase+=1   

/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/2-dimensional/../common/loo_cv.py:17: RuntimeWarning: divide by zero encountered in divide
  error_vector = np.divide(w,np.diagonal(invM))


20 & 1.4e-01 & 1.7e-01 & 1.7e-01                & 1.6e+01 & 6.9e-01  \\ 


/users/PAS2708/hanveiga1/RBF-shape-parameter-NN/2-dimensional/../common/loo_cv.py:17: RuntimeWarning: invalid value encountered in divide
  error_vector = np.divide(w,np.diagonal(invM))


40 & 5.4e-01 & 6.6e-01 & 6.5e-01                & 6.2e+01 & 2.6e+00  \\ 
